# Setup

In [3]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import os
import cv2

from glob import glob
from pathlib import Path

In [4]:
pip install wolta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------  1.0/1.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------  1.6/1.6 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
    --------------------------------------- 3.7/241.3 MB 19.8 MB/s eta 0:00:12
   - -------------------------------------- 10.7/241.3 MB 25.8 MB/s eta 0:00:09
   --- ------------------------------------ 19.1/241.3 MB 30.2 MB/s eta 0:00:08
   ---- ----------------------------------- 27.0/241.3 MB 32.3 MB/s eta 0:00:07
   ----- ---------------------------------- 30.9/241.3 MB 31.2 MB/s eta 0:00:07
   ----- 


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
os.environ['WANDB_MODE'] = 'disabled'

# Data Analysis

In [7]:
for dirname, _, _ in os.walk('/kaggle/input'):
    print(dirname)

In [8]:
p_paths = glob('/kaggle/input/fondo-del-ojo-normal-glaucoma-retinopatia/dataset/*')

p_paths

[]

In [9]:
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))

d_paths

[]

In [10]:
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

len(i_paths)

0

In [11]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{}

In [ ]:
from wolta.visual_tools import dataset_size_same
import glob

# path to your dataset folder
dataset_folder = r"C:\Users\Saif\OneDrive\Desktop\Glaucoma Detection\dataset\train\normal"

# collect all .jpg files
i_paths = glob.glob(dataset_folder + "/*.jpg")

print("Found", len(i_paths), "images")
print(i_paths[:5])  # show first 5 image paths


0
[]


In [ ]:
from wolta.visual_tools import dataset_ratio_same

dataset_ratio_same(i_paths)

False

# Data Manipulation

## Image Processing & Stacking

In [ ]:
os.makedirs('/kaggle/working/raw')

In [ ]:
from wolta.visual_tools import crop

for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = '/kaggle/working/raw/{}'.format(current_dir)
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))
    for i_path in i_paths:
        current_img = Path(i_path).name
        img = cv2.imread(i_path)

        edge = min(img.shape[0], img.shape[1])
        img = crop(img, crop_width=edge, crop_height=edge, get_img=True)
        img = cv2.resize(img, (128, 128))

        cv2.imwrite('{}/{}'.format(current_path, current_img), img)

## Data Split

In [ ]:
from wolta.visual_tools import dir_split

dir_split('/kaggle/working/raw', '/kaggle/working/data', test_size=0.2, val_size=0.2)

## Path Spec

In [ ]:
parent_dir = '/kaggle/working/data'
train_dir = '/kaggle/working/data/train'
test_dir = '/kaggle/working/data/test'
val_dir = '/kaggle/working/data/val'

# Model

## YOLO

In [ ]:
from ultralytics import YOLO

model = YOLO(model='yolo11x-cls.pt')
results = model.train(data=parent_dir, epochs=5, imgsz=128)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 56.9M/56.9M [00:00<00:00, 183MB/s]


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.pt, data=/kaggle/working/data, epochs=5, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 5.35M/5.35M [00:00<00:00, 70.0MB/s]


AMP: checks passed ✅


...

               classes   top1_acc   top5_acc: 100%|██████████| 114/114 [00:03<00:00, 34.77it/s]

                   all       0.88          1



5 epochs completed in 0.078 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 57.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 57.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 10889 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 3628 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 3628 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 114/114 [00:03<00:00, 36.60it/s]


                   all       0.88          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


## CNN

### Data Preparation

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',   
    label_mode='int',
    batch_size=16,
    image_size=(128, 128),
    shuffle=True 
)

Found 10889 files belonging to 2 classes.


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',   
    label_mode='int',
    batch_size=16,
    image_size=(128, 128),
    shuffle=True 
)

Found 3628 files belonging to 2 classes.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',   
    label_mode='int',
    batch_size=16,
    image_size=(128, 128),
    shuffle=True 
)

Found 3628 files belonging to 2 classes.


In [ ]:
names = train_ds.class_names
num_classes = len(names)

print(names)

['glaucoma', 'normal']


### Prediction

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential([
    layers.Input(shape=[128, 128, 3]),
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16384)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       2,097,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,121,122 (8.09 MB)

 Trainable params: 2,121,122 (8.09 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5
681/681 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.6514 - loss: 0.6050 - val_accuracy: 0.7552 - val_loss: 0.5104
Epoch 2/5
681/681 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7735 - loss: 0.4914 - val_accuracy: 0.8131 - val_loss: 0.4175
Epoch 3/5
681/681 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8058 - loss: 0.4271 - val_accuracy: 0.8346 - val_loss: 0.3934
Epoch 4/5
681/681 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8222 - loss: 0.3975 - val_accuracy: 0.8288 - val_loss: 0.3874
Epoch 5/5
681/681 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8288 - loss: 0.3842 - val_accuracy: 0.8401 - val_loss: 0.3750


In [ ]:
loss, acc = model.evaluate(test_ds)

print(f"Test Accuracy: {acc * 100:.2f}%")

227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8314 - loss: 0.3834
Test Accuracy: 84.04%
